# Convolutional Neural Networks (CNNs)

**Convolutional Neural Networks** vormen de ruggengraat van moderne computer vision. Ze zijn specifiek ontworpen om efficiënt om te gaan met de grid-achtige structuur van beeldgegevens en kunnen hiërarchische patronen leren van lokale beeldregio's.

## Model Architectuur

CNNs bestaan typisch uit verschillende lagen die elk een specifieke functie vervullen:

### Convolutionele Lagen

De **kerncomponent** van CNNs zijn de convolutionele lagen die **filters** (kernels) gebruiken om lokale patronen te detecteren:

```python
# Illustratie van een convolutionele operatie
import torch
import torch.nn as nn

# Convolutionele laag: 3 input kanalen, 64 output filters, 3x3 kernel
conv_layer = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
```

### Activatie Functies

**ReLU (Rectified Linear Unit)** is de meest gebruikte activatie functie:

$$f(x) = \max(0, x)$$

### Pooling Lagen

**Max pooling** reduceert de spatial dimensies terwijl belangrijke informatie behouden blijft:

```python
# Max pooling laag
pool_layer = nn.MaxPool2d(kernel_size=2, stride=2)
```

### Fully Connected Lagen

Aan het einde van het netwerk vertalen **dense lagen** de geleerde features naar de gewenste output.

## Parameters

CNN parameters bestaan uit:

- **Filter weights**: De convolutionele kernels
- **Biases**: Offset termen voor elke output feature map
- **Batch normalization parameters**: Schaal- en verschuivingsparameters
- **Dense layer weights**: Gewichten voor de classificatie lagen

### Parameter Delen

Een cruciaal aspect van CNNs is **parameter sharing** - dezelfde filters worden gebruikt over de gehele beeld:

```python
# Voor een 3x3 filter zijn er 3×3×3 = 27 parameters per filter
# Deze worden gedeeld over alle posities in de afbeelding
shared_params = 27  # per filter
```

## Features

CNNs extraheren automatisch hiërarchische features:

### Lage Niveaus
- **Edges en texturen**
- **Eenvoudige patronen**
- **Kleurcontrasten**

### Hoge Niveaus
- **Objectonderdelen**
- **Complexe structuren**
- **Semantische concepten**

### Translation Invariantie

CNNs zijn van nature **translation invariant** - ze herkennen patronen ongeacht hun positie in het beeld.

## Optimalisatie

### Backpropagation in CNNs

Het leerproces gebruikt **gradient descent** om de filters te optimaliseren:

```python
# Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()  # Backpropagation
    optimizer.step()
```

### Gradient Flow

De gradienten stromen terug door het netwerk om alle parameters bij te werken.

## Taken en Ervaring

CNNs worden gebruikt voor **supervised learning** taken:

- **Image classification**
- **Object detection**
- **Semantic segmentation**

### Data Annotatie

Training vereist **grote hoeveelheden gelabelde data**:
- Handmatige annotatie door experts
- Crowd-sourcing platforms
- Automatische labeling technieken

## Performantie

### Evaluatie Metrieken

- **Accuracy**: Percentage correcte voorspellingen
- **Precision & Recall**: Balans tussen false positives en false negatives
- **F1-score**: Harmonisch gemiddelde van precision en recall

### Computationele Kost

CNNs vereisen:
- **GPU acceleratie** voor training
- **Grote hoeveelheden geheugen**
- **Parallelle berekening** voor efficiëntie

## Voordelen

- **Automatische feature extractie**
- **Translation invariantie**
- **Hiërarchische representatie learning**
- **End-to-end training**
- **Herbruikbaarheid via transfer learning**

## Nadelen

- **Grote hoeveelheden trainingsdata nodig**
- **Computationeel intensief**
- **Moeilijk te interpreteren** (black box)
- **Gevoelig voor adversarial examples**
- **Overfitting bij kleine datasets**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

# Eenvoudige CNN voor MNIST cijfers
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # Convolutionele lagen
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        
        # Pooling lagen
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected lagen
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Dropout voor regularisatie
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # Convolution + ReLU + Pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # Flatten voor dense lagen
        x = x.view(-1, 64 * 7 * 7)
        
        # Dense lagen met dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Model initialisatie
model = SimpleCNN()
print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

# Visualisatie van de model architectuur
print("\nModel Architectuur:")
print(model)

## Feature Map Visualisatie

Laten we kijken hoe de feature maps evolueren door de CNN lagen:

In [ ]:
import torchvision
from torchvision import transforms

# Laad een voorbeeld afbeelding (MNIST stijl)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Maak een dummy afbeelding voor illustratie
dummy_input = torch.randn(1, 1, 28, 28)

# Visualisatie functie
def visualize_feature_maps(model, input_tensor, layer_name):
    """Visualiseer feature maps voor een specifieke laag"""
    model.eval()
    
    # Hook om feature maps te capturen
    feature_maps = []
    def hook_fn(module, input, output):
        feature_maps.append(output.detach())
    
    # Registreer hook op de gewenste laag
    if layer_name == 'conv1':
        model.conv1.register_forward_hook(hook_fn)
    elif layer_name == 'conv2':
        model.conv2.register_forward_hook(hook_fn)
    
    # Forward pass
    with torch.no_grad():
        output = model(input_tensor)
    
    return feature_maps[0] if feature_maps else None

# Genereer feature maps
conv1_features = visualize_feature_maps(model, dummy_input, 'conv1')
conv2_features = visualize_feature_maps(model, dummy_input, 'conv2')

print(f"Conv1 feature maps shape: {conv1_features.shape}")
print(f"Conv2 feature maps shape: {conv2_features.shape}")

# Plot eerste paar feature maps
fig, axes = plt.subplots(2, 4, figsize=(12, 6))

for i in range(4):
    # Conv1 feature maps
    axes[0, i].imshow(conv1_features[0, i].cpu().numpy(), cmap='viridis')
    axes[0, i].set_title(f'Conv1 - Filter {i+1}')
    axes[0, i].axis('off')
    
    # Conv2 feature maps
    axes[1, i].imshow(conv2_features[0, i].cpu().numpy(), cmap='viridis')
    axes[1, i].set_title(f'Conv2 - Filter {i+1}')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()